In [1]:
import open3d as o3d
from PIL import Image
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
from pathlib import Path

%matplotlib inline

In [2]:
path = Path("output2/")

In [3]:
# general images
K,col, disp_col, disp = [],[],[],[]
f = []
files = list(path.glob("*.npz"))
for path_tmp in files:
    arr = np.load(str(path_tmp.with_suffix(".npz")))
    name = path_tmp.stem
    try:
        img = Image.open(str(path_tmp.parent.parent / "media/tests" / (path_tmp.stem + ".png")))
        img = img.resize(arr['depth'].shape)
    except:
        continue
    disp.append(arr)
    col.append(np.array(img))
    f.append(path_tmp)

In [4]:
rgb, xyz = [], []
for num in range(len(col)):
    #intrinsic = np.eye(3)
    #intrinsic[0,0] = K[num][0,0]
    #intrinsic[1,1] = K[num][0,1]
    #intrinsic[0,2] = K[num][0,2]
    #intrinsic[1,2] = K[num][0,3]
    
    xx, yy = np.meshgrid(range(col[num].shape[1]), range(col[num].shape[0]))

    rgb.append(col[num].reshape((-1,3)) / 255)
    zz = disp[num]['depth'].flatten()
    xx = xx.flatten() / xx.max() * 2 - 1
    yy = yy.flatten() / yy.max()
    ones = np.ones_like(yy)

    p = np.vstack((xx,yy,ones)).transpose((1,0)) * zz[:,None]
    #p = (np.linalg.inv(intrinsic) @ p[:,:,None])[:,:,0]
    xyz.append(p)

In [5]:
pcds = []
for num in range(len(col)):
    pcd = o3d.PointCloud()
    pcd.points = o3d.Vector3dVector(xyz[num])
    pcd.colors = o3d.Vector3dVector(rgb[num])
    pcds.append(pcd)

In [8]:
for i,num in enumerate(f):
    o3d.write_point_cloud("output2/%s.ply" % num.stem, pcds[i])

In [ ]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6, origin=[0, 0, 0])
o3d.visualization.draw_geometries([mesh_frame, pcds[1]])

In [ ]:
sorted(disp[2])

In [ ]:
disp[2]['depth'].shape